In [1]:
import pandas as pd
import numpy as np
from utils import graphics
import plotly.express as px
import talib
import seaborn as sns

In [31]:
dataset = pd.read_csv('../data/GALA.csv', sep=';')

dataset = dataset.drop(['Ignore', 'symbol', 'CloseTime'], axis=1)
dataset['OpenTime'] = pd.to_datetime(dataset['OpenTime'] / 1000, unit='s')

dataset['Date'] = pd.DatetimeIndex(dataset['OpenTime']).date
dataset['Date'] = dataset['Date'].astype('str')
dataset['Day'] = dataset['OpenTime'].dt.day
dataset['Year'] = pd.DatetimeIndex(dataset['OpenTime']).year
dataset['Month'] = pd.DatetimeIndex(dataset['OpenTime']).month
dataset['Hour'] = pd.DatetimeIndex(dataset['OpenTime']).hour

print(f'Dataset shape: {dataset.shape}')

Dataset shape: (106081, 16)


In [32]:
dataset['QuantityForDay'] = dataset.groupby(['Date'])['Close'].transform('count')
dataset = dataset[dataset['QuantityForDay'] == 1440]

dataset = dataset.reset_index()
dataset.head()

,index,OpenTime,Open,High,Low,Close,Volume,QuoteAssetVolume,NumberTraders,TakerBuyBase,TakerBuyQuote,ID,Date,Day,Year,Month,Hour,QuantityForDay
0,1080,2021-09-14 00:00:00,0.04721,0.04725,0.04713,0.04718,870455,41091.43894,140,661715,31241.83314,18126097,2021-09-14,14,2021,9,0,1440
1,1081,2021-09-14 00:01:00,0.04719,0.04720,0.04688,0.04692,4210823,198065.43074,187,1483187,69722.26816,18126098,2021-09-14,14,2021,9,0,1440
2,1082,2021-09-14 00:02:00,0.04688,0.04689,0.04650,0.04668,7484820,349275.19481,346,5253995,245104.81032,18126099,2021-09-14,14,2021,9,0,1440
3,1083,2021-09-14 00:03:00,0.04666,0.04700,0.04631,0.04632,3110185,145070.61671,286,852687,39836.51348,18126100,2021-09-14,14,2021,9,0,1440
4,1084,2021-09-14 00:04:00,0.04638,0.04652,0.04629,0.04646,1083207,50234.61260,157,384166,17827.07697,18126101,2021-09-14,14,2021,9,0,1440


### Features

In [46]:
dataset['HighDiff1'] = dataset['High'].diff()
dataset['LowDiff1'] = dataset['Low'].diff()
dataset['OpenDiff1'] = dataset['Open'].diff()
dataset['CloseDiff1'] = dataset['Close'].diff()

dataset['HighLow'] = dataset['High'] - dataset['Low']
dataset['OpenClose'] = dataset['Close'] - dataset['Open']
dataset['OpenCloseMean'] = ((dataset['Close'] + dataset['Open']) / 2)
dataset['HighDiffOpenClose'] = dataset['High'] - dataset['OpenCloseMean']
dataset['LowDiffOpenClose'] = dataset['OpenCloseMean'] - dataset['Low']

In [30]:
def show_graphics_levels():

    for i in [3, 5, 15, 30]:

        group = dataset.groupby(dataset.index // i).agg({
        'OpenTime': 'last',
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum',
        'NumberTraders': 'sum'
        })

        cant_show = int((60 / i) * 4)

        graphics.candelstick_chart(group[-cant_show:], '')

show_graphics_levels()

In [47]:
px.line(dataset[-120:], x='OpenTime', y=['HighDiffOpenClose', 'LowDiffOpenClose']).show()

In [ ]:
upper, middle, lower = talib.BBANDS(dataset.Close, matype=talib.MA_Type.MA_Type.T3)
